In [1]:
from datasets import load_dataset, load_from_disk
dataset = load_dataset("cais/mmlu", "abstract_algebra")

In [5]:
dataset['test'][0]

{'question': 'Find the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q.',
 'subject': 'abstract_algebra',
 'choices': ['0', '4', '2', '6'],
 'answer': 1}

In [2]:
dataset = load_from_disk('final_dataset')

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'response'],
        num_rows: 18396
    })
    validation: Dataset({
        features: ['prompt', 'response'],
        num_rows: 3563459
    })
})